In [1]:
import numpy as np
# import ahrs
# import scipy
from scipy.linalg import expm #, sinm, cosm

import pygame
import matplotlib.pyplot as plt

import quaternion

pygame 2.6.1 (SDL 2.28.4, Python 3.13.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
""" Cube Visualisation: Setup and function definitions """

pygame.init()

WIDTH, HEIGHT = 1280, 720

WHITE = (255, 255, 255)
L_RED = (255, 200, 150)
L_GREEN = (150, 255, 150)
L_BLUE = (150, 200, 255)
RED   = (255, 0, 0)
GREEN = (0, 255, 0)
CYAN  = (0, 100, 255)
BLACK = (40, 40, 40)

font = pygame.font.Font('freesansbold.ttf', 32)
label_true = font.render('MEKF2', True, L_RED, BLACK)
label_hat = font.render('MEKF2 Rm>Ra', True, L_GREEN, BLACK)
labelRect_true = label_true.get_rect()
labelRect_hat = label_hat.get_rect()
labelRect_true.center = (.25*WIDTH, .1*HEIGHT)
labelRect_hat.center = (.75*WIDTH, .1*HEIGHT)

c_cube_true = [.25*WIDTH, 3*HEIGHT/5]  # x, y
c_cube_hat = [.75*WIDTH, 3*HEIGHT/5]  # x, y

points = []

# all the cube vertices
points.append(np.matrix([-1, -1, .4]))
points.append(np.matrix([1, -1, .4]))
points.append(np.matrix([1,  1, .4]))
points.append(np.matrix([-1, 1, .4]))
points.append(np.matrix([-1, -1, -.4]))
points.append(np.matrix([1, -1, -.4]))
points.append(np.matrix([1, 1, -.4]))
points.append(np.matrix([-1, 1, -.4]))


proj_points_true = [
    [n, n] for n in range(len(points))
]
proj_points_hat = [
    [n, n] for n in range(len(points))
]

def connect_points(screen, i, j, points, color):
    pygame.draw.line(
        screen, color, (points[i][0], points[i][1]), (points[j][0], points[j][1]), 3)

def get_proj_xy(DCM, point, center, scale, distance):
    rotated2d  = np.dot(DCM, point.reshape((3, 1)))
    x = float(rotated2d[0][0].item())
    y = float(rotated2d[1][0].item())
    z = float(rotated2d[2][0].item())
    # print(str(int(x)) + " " + str(int(y)) + " " + str(int(z)))
    rotated2d[0][0] = y
    rotated2d[1][0] = -z
    rotated2d[2][0] = -x
    
    z = 1/(distance - float(rotated2d[2][0].item()))
    projection_matrix = np.matrix([
        [z, 0, 0],
        [0, z, 0]
    ])
    projected2d = np.dot(projection_matrix, rotated2d)

    x = int(projected2d[0][0].item() * scale *distance) + center[0]
    y = int(projected2d[1][0].item() * scale *distance) + center[1]

    return [x, y]


In [3]:
""" Simulation Setup """

### Simulation duration and time step
duration = 10.0         # seconds
dt = 0.01               # time step in seconds 100FPS=.01sec/frame
N = int(duration / dt)  # 10/0.01 = 1000

### Motion Setup
# R = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])          # initial configuration of the system 
R = np.matrix([
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ])
omega = np.array([-1, -1, 0])
Omega = np.array([[0, -1,  1], [1, 0, 0], [-1, 0, 0]])  # constant control input - rad/sec

### Sensor Setup
a_ref = np.array([0, 0, 1])         # NED
m_ref = np.array([.877, 0, .4796])  # NED
                        # WMM data for LatLong = 19.1334° N, 72.9133° E (Mumbai)
                        # Declination (+E|-W)  = Negligible: -0.0477° or .05W  +- 0.30°
                        # Inclination (+D|-U)  = 28.6536° +- 0.21°
gyro = Omega
acc = np.matmul(R, a_ref)

##### Sensor characteristic 
mu_g = [0, 0, 0]    #rad/sec
cov_g = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]  
mu_a = [0, 0, 0]
cov_a = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
# mu_g = [0, 0, 0]
# cov_g = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]




In [4]:
""" Estimator Setup """ 
R_hat = np.matrix([
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, 1],
    ])

# Estimator Evaluation: Variables for Plots
time = np.linspace(0, duration, N)
err = []





In [ ]:
""" Run: Simulation, Estimation, Cube-Visualisation and Collect Data """

err = []

pygame.display.set_caption("3D projection in pygame!")
screen = pygame.display.set_mode((WIDTH, HEIGHT))

clock = pygame.time.Clock()

for n in range(N):

    clock.tick(100)
    for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    pygame.quit()
                    exit()

    # Simulation
    R = np.matmul(R,expm(Omega*dt))

    # Take Measurement
    gyro = Omega + np.random.multivariate_normal(mu_g, cov_g)
    acc = np.matmul(R, a_ref) + np.random.multivariate_normal(mu_a, cov_a)
    print(gyro)

    # Record Data
    err.append(3-np.trace(np.matmul(R,np.transpose(R_hat))));
    # Visualise Cubes
    i = 0
    screen.fill(BLACK)

    for point in points:

        proj_points_true[i]  = get_proj_xy(R, point, c_cube_true, 150, 50)
        proj_points_hat[i] = get_proj_xy(R_hat, point, c_cube_hat, 200, 50)

        # print(float(point.reshape((3, 1))[0]))
        if float(point.reshape((3, 1))[0].item()) < 0:
            pygame.draw.circle(screen, RED, (proj_points_true[i][0], proj_points_true[i][1]), 5)
            pygame.draw.circle(screen, RED, (proj_points_hat[i][0], proj_points_hat[i][1]), 5)
        else:
            pygame.draw.circle(screen, GREEN, (proj_points_true[i][0], proj_points_true[i][1]), 5)
            pygame.draw.circle(screen, GREEN, (proj_points_hat[i][0], proj_points_hat[i][1]), 5)
        i += 1

    for p in range(4):
        connect_points(screen, p, (p+1) % 4, proj_points_true, L_RED)
        connect_points(screen, p+4, ((p+1) % 4) + 4, proj_points_true, L_RED)
        connect_points(screen, p, (p+4), proj_points_true, L_RED)
        
        connect_points(screen, p, (p+1) % 4, proj_points_hat, L_GREEN)
        connect_points(screen, p+4, ((p+1) % 4) + 4, proj_points_hat, L_GREEN)
        connect_points(screen, p, (p+4), proj_points_hat, L_GREEN)
    
    # Labels

    screen.blit(label_true, labelRect_true)
    screen.blit(label_hat, labelRect_hat)

    pygame.display.update()

pygame.quit()

""" Plot the Results """

plt.plot(time, err)
plt.show()


[[ 0.51512882 -1.34665371  0.4083222 ]
 [ 1.51512882 -0.34665371 -0.5916778 ]
 [-0.48487118 -0.34665371 -0.5916778 ]]
[[ 0.16787061 -1.78102608 -0.19526552]
 [ 1.16787061 -0.78102608 -1.19526552]
 [-0.83212939 -0.78102608 -1.19526552]]
[[ 0.93019542 -0.9781982   1.98607619]
 [ 1.93019542  0.0218018   0.98607619]
 [-0.06980458  0.0218018   0.98607619]]
[[ 1.33992677 -1.34789164  0.99471528]
 [ 2.33992677 -0.34789164 -0.00528472]
 [ 0.33992677 -0.34789164 -0.00528472]]
[[ 0.32834823 -1.44624131  1.58104554]
 [ 1.32834823 -0.44624131  0.58104554]
 [-0.67165177 -0.44624131  0.58104554]]
[[-0.0989949  -0.50713491  0.202844  ]
 [ 0.9010051   0.49286509 -0.797156  ]
 [-1.0989949   0.49286509 -0.797156  ]]
[[ 0.04980065 -1.66656046  0.53357918]
 [ 1.04980065 -0.66656046 -0.46642082]
 [-0.95019935 -0.66656046 -0.46642082]]
[[ 0.23321247 -0.78687347  1.97545152]
 [ 1.23321247  0.21312653  0.97545152]
 [-0.76678753  0.21312653  0.97545152]]
[[ 2.09564386 -2.13507446  1.99177541]
 [ 3.09564386 -1.

error: display Surface quit

: 